In [5]:
#read the csv file
import pandas as pd
can = pd.read_csv("https://raw.githubusercontent.com/medcab9/ds/master/data/Cannabis%20StrainsFeatures%20-%20cannabis.csv")
print(can.shape)
can.head()

(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [6]:
can.describe(include='all')

,Strain,Type,Rating,Effects,Flavor,Description
count,2351,2351,2351.000000,2351,2350,2350
unique,2350,3,NaN,1704,1349,2343
top,B-Witched,hybrid,NaN,None,None,None
freq,2,1212,NaN,29,44,5
mean,NaN,NaN,4.382561,NaN,NaN,NaN
std,NaN,NaN,0.639090,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,4.200000,NaN,NaN,NaN
50%,NaN,NaN,4.500000,NaN,NaN,NaN
75%,NaN,NaN,4.700000,NaN,NaN,NaN


In [7]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         1
Description    1
dtype: int64

In [8]:
# Cleaning the data and changing nan values to unknowns and 0s
import numpy as np

can['Rating'] = can['Rating'].replace(np.nan, 0)
can['Effects'] = can['Effects'].replace(np.nan, 'Effects currently unknown')
can['Flavor'] = can['Flavor'].replace(np.nan, 'Flavor profile currently unavailable')
can['Description'] = can['Description'].replace(np.nan, 'Strain description currently unavailable')
can['Type'] = can['Type'].replace(np.nan, 'Type currently unknown')


In [9]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

In [10]:
# Unique Strain types

can.Strain.value_counts()

B-Witched            2
Pure-Afghan          1
Blackberry-Kush      1
Ganesh-Berry         1
Tahoe-Frostbite      1
                    ..
Watermelon           1
Pre-98-Bubba-Kush    1
Snow-Mountain        1
Juicy-Fruit          1
Scooby-Snack         1
Name: Strain, Length: 2350, dtype: int64

In [11]:
can['Rating'].unique()

array([4. , 4.7, 4.4, 4.2, 4.6, 4.5, 4.3, 5. , 3.8, 4.1, 4.8, 3.4, 3.7,
       3.9, 0. , 3. , 4.9, 3.6, 2.8, 3.3, 3.5, 3.2, 1. , 3.1, 2.5])

In [12]:
# Create new series with information

can['total_text'] = can['Strain'] + can['Effects'] + can['Description']+can["Flavor"]

In [13]:
can.head()

,Strain,Type,Rating,Effects,Flavor,Description,total_text
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat..."


## 2) Use Spacy to tokenize the total text

In [12]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=1b30e2abcbd88a2300ff2e70305d02f557efde7c104fad3491b6688cadf10216
  Stored in directory: /tmp/pip-ephem-wheel-cache-un8l43dz/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [14]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer
from bs4 import BeautifulSoup

In [15]:
# Inialize spaCy model and tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [16]:
tokens = []

# Return lower case tokens and ignore stop words/punctuation
for doc in tokenizer.pipe(can['total_text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        # ignore stopwords, spaces and punctuation
        if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

can['spaCy_tokens'] = tokens
can['spaCy_tokens'].head()

0    [100-ogcreative,energetic,tingly,euphoric,rela...
1    [98-white-widowrelaxed,aroused,creative,happy,...
2    [1024uplifted,happy,relaxed,energetic,creative...
3    [13-dawgstingly,creative,hungry,relaxed,uplift...
4    [24k-goldhappy,relaxed,euphoric,uplifted,talka...
Name: spaCy_tokens, dtype: object

In [17]:
print(can.shape)
can.head()

(2351, 8)


,Strain,Type,Rating,Effects,Flavor,Description,total_text,spaCy_tokens
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax...","[100-ogcreative,energetic,tingly,euphoric,rela..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E...","[98-white-widowrelaxed,aroused,creative,happy,..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1...","[1024uplifted,happy,relaxed,energetic,creative..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte...","[13-dawgstingly,creative,hungry,relaxed,uplift..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat...","[24k-goldhappy,relaxed,euphoric,uplifted,talka..."


## Part 2: Vector Representation

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer

In [19]:
def gather_data(df_column):
    """ Produces List of Documents from a df column    
    returns list of strings 
    """
    
    #data = []
    
#     for row in df_column:
#         data.append(' '.join(str(row))
    data = [''.join(str(item)) for item in df_column]
        
    
    return data

In [20]:
data = gather_data(can['spaCy_tokens'])
data[0]

"['100-ogcreative,energetic,tingly,euphoric,relaxed$100', 'og', '50/50', 'hybrid', 'strain', 'packs', 'strong', 'punch.', 'supposedly', 'refers', 'strength', 'high', 'price', 'started', 'showing', 'hollywood.', 'plant,', '$100', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'buds', 'stems.', 'users', 'report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert,', 'cerebral', 'feeling', 'thanks', 'sativa', 'side.earthy,sweet,citrus']"

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# create the transformer
vect = CountVectorizer()

# build vocab

vect.fit(data)

# transform text
dtm = vect.transform(data)
print(dtm.shape)

(2351, 12066)


In [52]:
# Tunning Parameters

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        ngram_range = (1,2),
                        max_df=.97,
                        min_df=3)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(data)


# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,09,09 cookies,10,10 11,10 feet,10 week,10 weeks,100,100 sativa,11,11 week,11 weeks,12,12 14,12 weeks,13,13 14,13 haze,13 weeks,14,14 cbd,14 week,14 weeks,15,15 20,15 cbd,16,16 cbd,17,18,18 24,19,1960s,1970s,1980s,1989,1990s,1991,1996,1997,...,wreckrelaxed,wreckuplifted,xiii,xxx,year,years,yellow,yeti,yield,yield attentive,yield average,yield flowering,yield potency,yield thc,yielders,yielding,yielding plant,yielding plants,yields,yields dense,yields following,yields frosty,yields high,yields indoors,yields potent,yields resinous,yields strain,yields sweet,yields thc,yoda,yunnan,zealand,zest,zestful,zesty,zesty lemon,zion,zion botanicals,zombie,zombie og
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.452944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10533,0.132443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating the model

In [56]:
from sklearn.neighbors import NearestNeighbors

#Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [57]:
# from sklearn.pipeline import make_pipeline
# from sklearn.neighbors import NearestNeighbors

# pipeline = make_pipeline(TfidfVectorizer(stop_words='english',
#                                          ngram_range = (1,2),
#                                          max_df=.97,
#                                          min_df=3),
#                                          NearestNeighbors(n_neighbors=5, algorithm='kd_tree'))



# pipeline.fit(can['total_text'])

In [58]:
# get the top 5 nearest neighbors
nn.kneighbors([dtm.iloc[0].values])

(array([[0.        , 1.28254142, 1.30162916, 1.31681204, 1.3181531 ]]),
 array([[   0,  682,  959,  237, 1891]]))

In [59]:
# Write out fake 


fake_input= [" I am looking for something which helps with my anxiety  and helps me go to sleep. I like it to have fruity flavour "]

print(fake_input)

[' I am looking for something which helps with my anxiety  and helps me go to sleep. I like it to have fruity flavour ']


In [60]:
new = tfidf.transform(fake_input)
new

<1x9694 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [61]:
nn.kneighbors(new.todense())

(array([[1.30403635, 1.30786077, 1.3129059 , 1.32636415, 1.32697429]]),
 array([[ 411, 1889,  343, 2318,  153]]))

In [62]:
# Print two results

print("Recommendation 1---",can['total_text'][411])

print("Recommendation 2---",can['total_text'][1889])

Recommendation 1--- ButterscotchRelaxed,Euphoric,Focused,Creative,SleepyButterscotch is a treat of a strain that gets its name from its distinctive candy aroma. This strong indica helps patients to gain some mental peace and unwind from a stressful day. Effects tend to be sedative and sleep-inducing, so Butterscotch is a good choice for treating insomnia.Pine,Citrus,Nutty
Recommendation 2--- SkywalkerRelaxed,Euphoric,Sleepy,Happy,UpliftedSkywalker is a well-rounded indica-dominant hybrid that helps patients knock out pain and relax into sleep after a long day of battling the Dark Side. The sativa side of this strain allows for a nice head high as well, but it isn’t overly hazy or unfocused. Skywalker was created by Amsterdam’s Dutch Passion Seeds to combine the sweet fruity aroma of Blueberry with the growing strengths of the hardy, high-yielding Mazar. Great for indoor growing, Skywalker plants will stay shorter and bushy, flowering in 8 to 9 weeks. If you’re looking for the Californi

In [63]:
response = can[['Strain','Type', "Rating",'Effects', 'Flavor', 'Description']].iloc[[411]]
response

,Strain,Type,Rating,Effects,Flavor,Description
411,Butterscotch,indica,4.3,"Relaxed,Euphoric,Focused,Creative,Sleepy","Pine,Citrus,Nutty",Butterscotch is a treat of a strain that gets ...


In [64]:
response.to_json(orient='records')

'[{"Strain":"Butterscotch","Type":"indica","Rating":4.3,"Effects":"Relaxed,Euphoric,Focused,Creative,Sleepy","Flavor":"Pine,Citrus,Nutty","Description":"Butterscotch is a treat of a strain that gets its name from its distinctive candy aroma. This strong indica helps patients to gain some mental peace and unwind from a stressful day. Effects tend to be sedative and sleep-inducing, so Butterscotch is a good choice for treating insomnia."}]'

In [ ]:
# import pickle

# with open('knn_model.pkl', 'wb') as f:
#     pickle.dump(nn, f)

In [ ]:
# # Instantiate vectorizer object
# tfidf = TfidfVectorizer(stop_words='english',
#                         ngram_range = (1,2),
#                         max_df=.97,
#                         min_df=3)

# # Create a vocabulary and get word counts per document
# dtm = tfidf.fit_transform(input)


# # Get feature names to use as dataframe column headers
# dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# # View Feature Matrix as DataFrame
# dtm.head()

In [ ]:
# pickle.dump(tfidf, open("vectorizer.pickle", "wb"))

In [ ]:
## have to run the code block
## have to tokenize the input before vectorizing it

In [65]:
import pandas as pd
df = pd.read_json (r'Path where the JSON file is saved\File Name.json')
in = df.to_csv (r'Path where the new CSV file will be stored\New File Name.csv', index = None)

SyntaxError: ignored

In [ ]:
in['total_text'] = in['Strain'] + in['Effects'] + in['Description']+ in["Flavor"]

In [ ]:
# Inialize spaCy model and tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
tokens = []
# Return lower case tokens and ignore stop words/punctuation
for doc in tokenizer.pipe(in['total_text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        # ignore stopwords, spaces and punctuation
        if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

in['spaCy_tokens'] = tokens

In [ ]:
vector = load(open(vectorizer.pickle, 'rb'))

In [ ]:
vector_t = vector.transform(in['spacy_tokens'])

In [ ]:
model = load(open(knn_model.pkl), 'rb')

In [ ]:
model.vector_t